In [1]:
import numpy as np
import pandas as pd
import json
import collections
from collections import defaultdict 
from functools import partial
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import jax

# Global flag to set a specific platform, must be used at startup.
jax.config.update('jax_platform_name', 'gpu')
jax.config.update("jax_debug_nans", True)
jax.config.update("jax_debug_infs", True)

jax.config.update('jax_log_compiles', False)
jax.config.update('jax_check_tracer_leaks', False)

In [2]:
# Good read: https://iq-inc.com/importerror-attempted-relative-import/

import sys
import importlib
from mimicnet import concept
from mimicnet import jax_interface
from mimicnet import dag
from mimicnet import glove
from mimicnet import gram
from mimicnet import ode

importlib.reload(sys.modules['mimicnet.concept'])
importlib.reload(sys.modules['mimicnet.dag'])
importlib.reload(sys.modules['mimicnet.jax_interface'])
importlib.reload(sys.modules['mimicnet.glove'])
importlib.reload(sys.modules['mimicnet.gram'])
importlib.reload(sys.modules['mimicnet.ode'])

<module 'mimicnet.ode' from '/home/asem/GP/MIMIC-SNONET/mimicnet/ode.py'>

In [3]:

KG = dag.CCSDAG()

In [4]:
# multi_visit_mimic_dir = '/home/am8520/GP/ehr-data/mimic3-multi-visit'
multi_visit_mimic_dir = '/home/asem/GP/ehr-data/mimic3-multi-visit'
transformed_mimic_dir = '/home/asem/GP/ehr-data/mimic3-transforms'
mimic_dir = '/home/asem/GP/ehr-data/mimic3-v1.4/physionet.org/files/mimiciii/1.4'
experiments_dir = '/home/asem/GP/ehr-data/mimic3-snonet-exp'
experiment_prefix = 'NOV29'

In [5]:
D_LABITEMS = pd.read_csv(f'{mimic_dir}/D_LABITEMS.csv.gz')
D_ITEMS = pd.read_csv(f'{mimic_dir}/D_ITEMS.csv.gz')
D_TEST = pd.concat([D_LABITEMS, D_ITEMS], join='inner')
test_label_dict = dict(zip(D_TEST.ITEMID, D_TEST.LABEL))
test_cat_dict = dict(zip(D_TEST.ITEMID, D_TEST.CATEGORY))

In [6]:
static_df = pd.read_csv(f'{transformed_mimic_dir}/static_df.csv.gz')
adm_df = pd.read_csv(f'{transformed_mimic_dir}/adm_df.csv.gz')
diag_df = pd.read_csv(f'{transformed_mimic_dir}/diag_df.csv.gz', dtype={'ICD9_CODE': str})
proc_df = pd.read_csv(f'{transformed_mimic_dir}/proc_df.csv.gz', dtype={'ICD9_CODE': str})
test_df = pd.read_csv(f'{transformed_mimic_dir}/test_df.csv.gz')


In [7]:
# Cast columns of dates to datetime64

static_df['DOB'] = pd.to_datetime(static_df.DOB, infer_datetime_format=True).dt.normalize()
adm_df['ADMITTIME'] = pd.to_datetime(adm_df.ADMITTIME, infer_datetime_format=True).dt.normalize()
adm_df['DISCHTIME'] = pd.to_datetime(adm_df.DISCHTIME, infer_datetime_format=True).dt.normalize()
test_df['DATE'] = pd.to_datetime(test_df.DATE, infer_datetime_format=True).dt.normalize()

In [8]:
static_df.ETHNIC_GROUP.nunique()

7

In [9]:
test_df

,SUBJECT_ID,ITEMID,DATE,VALUE
0,17,50852,2134-12-22,-1.201339
1,17,50861,2134-12-22,-0.809370
2,17,50862,2134-12-22,1.634410
3,17,50863,2134-12-22,-1.452596
4,17,50867,2134-12-22,-0.608894
...,...,...,...,...
4546204,99982,227456,2157-02-22,1.794272
4546205,99982,227457,2157-02-22,-0.723781
4546206,99982,227465,2157-02-22,2.247434
4546207,99982,227466,2157-02-22,-0.029395


In [10]:
patients = concept.Subject.to_list(static_df, adm_df, diag_df, proc_df, test_df)

In [11]:
len(patients)

4434

In [12]:
# ehr_dfs = concept.Subject.to_df(patients)

In [13]:
# def df_eq(df1, df2):
#     df11 = df1.sort_values(by=df1.columns.tolist()).reset_index(drop=True)
#     df22 = df2.sort_values(by=df2.columns.tolist()).reset_index(drop=True)
#     return df11[df22.columns].equals(df22)

# all(map(df_eq, ehr_dfs, [static_df, adm_df, diag_df, proc_df, test_df]))

### [FORK] Skip the cell below to load the jaxified data from a stored file on disc

In [14]:
subjects_interface = jax_interface.SubjectJAXInterface(patients, set(test_df.ITEMID), KG)
# import pickle
# with open(f'{experiments_dir}/{experiment_prefix}_subjects_interface.pkl', 'wb') as pickleFile:
#     pickle.dump(subjects_interface, pickleFile)

In [15]:
# import pickle
# with open(f'{experiments_dir}/{experiment_prefix}_subjects_interface.pkl', 'rb') as pickleFile:
#     subjects_interface = pickle.load(pickleFile)

## GloVe Initialization

In [16]:
%load_ext autoreload
%autoreload 2

In [17]:
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug("test")

In [18]:
glove_args = {
    'diag_idx': subjects_interface.diag_multi_ccs_idx,
    'proc_idx': subjects_interface.proc_multi_ccs_idx,
    'ccs_dag': KG,
    'subjects': patients,
    'diag_vector_size': 100,
    'proc_vector_size': 60,
    'iterations': 30,
    'window_size_days': 2 * 365
}

diag_glove_rep, proc_glove_rep = glove.glove_representation(**glove_args)

In [19]:
print(f'#point_indices: {len(subjects_interface.nth_points)}')
print(f'#total_points: {sum(len(points) for n, points in subjects_interface.nth_points.items())}')

#[len(points) for n, points in subjects_interface.nth_points.items()]

#point_indices: 1085
#total_points: 129334


## GRAM objects

In [20]:
from datetime import datetime
daily_tracer = "/tmp/tensorboard/"+ datetime.now().strftime("%Y%m%d-%H%M%S") 
print(daily_tracer)

/tmp/tensorboard/20211202-030520


In [21]:


logs = '/tmp/tensorboard/20210708-182059'
#server = jax.profiler.start_server(9999)

In [22]:
       
config = {
    'gram_config': {
        'diag': {
            'ccs_dag': KG,
            'code2index': subjects_interface.diag_multi_ccs_idx,
            'attention_method': 'tanh', #l2, tanh
            'attention_dim': 100,
            'ancestors_mat': subjects_interface.diag_multi_ccs_ancestors_mat,
            'basic_embeddings': diag_glove_rep
        },
        'proc': {
            'ccs_dag': KG,
            'code2index': subjects_interface.proc_multi_ccs_idx,
            'attention_method': 'tanh',
            'attention_dim': 60,
            'ancestors_mat': subjects_interface.proc_multi_ccs_ancestors_mat,
            'basic_embeddings': proc_glove_rep
        }
    },
    'model': {
        'ode_dyn': 'mlp', # gru, mlp
        'state_size': 50,
        'numeric_hidden_size': 40,
        'bias': True
    },
    'training': {
        'train_validation_split': 0.8,
        'batch_size': 8,
        'epochs': 1,
        'lr': 1e-3,
        'diag_loss': 'balanced_focal', # balanced_focal, bce
        'tay_reg': None, # Order of regularized derivative of the dynamics function (None for disable).
        'loss_mixing': {
            'num_alpha': 0.1,
            'diag_alpha': 0.1,
            'ode_alpha': 1e-3,
            'l1_reg': 1e-6,
            'l2_reg': 1e-5,
            'dyn_reg': 1e-5
        },
        'eval_freq': 1,
        'save_freq': 100,
        'save_params_prefix': None
    }
}


In [23]:
diag_gram = gram.DAGGRAM(**config['gram_config']['diag'])

In [24]:
proc_gram = gram.DAGGRAM(**config['gram_config']['proc'])

## GRU-ODE-Bayes

In [ ]:
import random
%load_ext autoreload
%autoreload 2
from absl import logging
logging.set_verbosity(logging.INFO)

#with jax.profiler.trace(logs):
res = ode.train_ehr(subject_interface=subjects_interface,
                diag_gram=diag_gram,
                proc_gram=proc_gram,
                rng=random.Random(42),
                **config['model'],
                **config['training'],
                verbose_debug=False,
                shape_debug=False,
                nan_debug=False,
                memory_profile=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO:ode:#params: 293044
info retrieval:   0%|          | 1/443 [01:21<10:03:43, 81.95s/it]                INFO:ode:
                        Training    Validation
prejump_num_loss       10.958753      9.129901
postjump_num_loss     689.882935    619.552429
prejump_diag_loss       0.001822      0.002045
postjump_diag_loss      0.001882      0.002085
num_loss               78.851171     70.172154
diag_loss               0.001828      0.002049
ode_loss                0.080677      0.072219
l1_loss             14239.277344  14239.277344
l1_loss_per_point      36.140298     53.733122
l2_loss              1785.047852   1785.047852
l2_loss_per_point       4.530578      6.736030
dyn_loss                 12632.0        8042.0
dyn_loss_per_week       9.621762     7.9872313
loss                 0.081056565    0.07260969
INFO:ode:
                          Training   Valdation
accuracy                  0.483177    0.470748
recall                    0.631579    0.616438
npv                       0

INFO:ode:
                          Training   Valdation
accuracy                  0.572983    0.564307
recall                    0.639640    0.617978
npv                       0.977260    0.977356
specificity               0.570528    0.562476
precision                 0.052015    0.045967
f1-score                  0.096206    0.085570
tp                        0.022727    0.020385
tn                        0.550256    0.543921
fp                        0.414213    0.423091
fn                        0.012804    0.012602
points_count            115.000000  189.000000
odeint_weeks_per_visit    9.115528    4.719577
nfe_per_point             0.000000    0.000000
nfe_per_week              0.000000    0.000000
nfex1000                  0.000000    0.000000
info retrieval:   2%|▏         | 7/443 [05:11<4:11:24, 34.60s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss       11.388617     10.092213
postjump_num_loss     702.533691    646.463318
preju

info retrieval:   3%|▎         | 12/443 [08:11<4:23:25, 36.67s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss        7.075703     12.469360
postjump_num_loss     443.473389    752.098572
prejump_diag_loss       0.001413      0.001495
postjump_diag_loss      0.001470      0.001481
num_loss               50.715471     86.432281
diag_loss               0.001418      0.001494
ode_loss                0.052132      0.087925
l1_loss             14227.782227  14227.782227
l1_loss_per_point      53.689744     74.103032
l2_loss              1766.709473   1766.709473
l2_loss_per_point       6.666828      9.201612
dyn_loss                  8156.0        5946.0
dyn_loss_per_week      30.432837      6.765605
loss                 0.052527267    0.08835449
INFO:ode:
                          Training   Valdation
accuracy                  0.661312    0.667253
recall                    0.716312    0.714286
npv                       0.986413    0.982198
spec

INFO:ode:
                          Training   Valdation
accuracy                  0.715330    0.711268
recall                    0.700000    0.793893
npv                       0.986116    0.988496
specificity               0.715845    0.707965
precision                 0.076433    0.098021
f1-score                  0.137818    0.174497
tp                        0.022752    0.030516
tn                        0.692579    0.680751
fp                        0.274919    0.280810
fn                        0.009751    0.007923
points_count            160.000000  329.000000
odeint_weeks_per_visit    4.841071    1.687364
nfe_per_point             0.000000    0.000000
nfe_per_week              0.000000    0.000000
nfex1000                  0.000000    0.000000
info retrieval:   4%|▍         | 18/443 [11:24<3:59:03, 33.75s/it]                INFO:ode:
                        Training    Validation
prejump_num_loss        9.422412      9.494567
postjump_num_loss     540.475647    535.555054
preju

...

info retrieval:  37%|███▋      | 166/443 [1:32:40<2:13:53, 29.00s/it]                INFO:ode:
                        Training    Validation
prejump_num_loss        8.601561      9.088069
postjump_num_loss     320.286926    345.487427
prejump_diag_loss       0.001004      0.001348
postjump_diag_loss      0.001015      0.001327
num_loss               39.770097     42.728005
diag_loss               0.001005      0.001346
ode_loss                0.040774      0.044072
l1_loss             13976.536133  13976.536133
l1_loss_per_point      98.426311     62.395251
l2_loss              1749.446289   1749.446289
l2_loss_per_point      12.320044      7.810028
dyn_loss                  4406.0        7132.0
dyn_loss_per_week      7.0852284      8.763209
loss                 0.041244738   0.044492308
INFO:ode:
                          Training   Valdation
accuracy                  0.786620    0.788292
recall                    0.817204    0.806061
npv                       0.992184    0.990807
sp

INFO:ode:
                          Training   Valdation
accuracy                  0.787559    0.790493
recall                    0.804348    0.785714
npv                       0.990802    0.988523
specificity               0.786932    0.790698
precision                 0.123539    0.138539
f1-score                  0.214182    0.235546
tp                        0.028951    0.032277
tn                        0.758607    0.758216
fp                        0.205399    0.200704
fn                        0.007042    0.008803
points_count            131.000000  210.000000
odeint_weeks_per_visit    4.193021    6.436735
nfe_per_point             0.000000    0.000000
nfe_per_week              0.000000    0.000000
nfex1000                  0.000000    0.000000
info retrieval:  39%|███▉      | 172/443 [1:35:32<2:10:20, 28.86s/it]                INFO:ode:
                        Training    Validation
prejump_num_loss       12.056873     13.214905
postjump_num_loss     457.232941    500.853760
pr

info retrieval:  40%|███▉      | 177/443 [1:38:28<2:12:54, 29.98s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss        9.508478     11.210136
postjump_num_loss     358.367767    424.205566
prejump_diag_loss       0.001475      0.000880
postjump_diag_loss      0.001502      0.000871
num_loss               44.394407     52.509679
diag_loss               0.001478      0.000879
ode_loss                0.045871      0.053388
l1_loss             13959.273438  13959.273438
l1_loss_per_point      33.157419     34.047008
l2_loss              1750.360840   1750.360840
l2_loss_per_point       4.157627      4.269173
dyn_loss                 13796.0       14608.0
dyn_loss_per_week      18.387663     16.334824
loss                 0.046252754     0.0537999
INFO:ode:
                          Training   Valdation
accuracy                  0.781338    0.788940
recall                    0.802575    0.851528
npv                       0.989295    0.990680
s

INFO:ode:
                          Training   Valdation
accuracy                  0.777729    0.769772
recall                    0.921053    0.846154
npv                       0.996477    0.994224
specificity               0.772769    0.767559
precision                 0.123023    0.095445
f1-score                  0.217054    0.171540
tp                        0.030810    0.023835
tn                        0.746919    0.745937
fp                        0.219630    0.225894
fn                        0.002641    0.004334
points_count            131.000000  351.000000
odeint_weeks_per_visit    2.351145    2.236874
nfe_per_point             0.000000    0.000000
nfe_per_week              0.000000    0.000000
nfex1000                  0.000000    0.000000
info retrieval:  41%|████▏     | 183/443 [1:42:22<2:39:28, 36.80s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss       13.026446      8.179623
postjump_num_loss     490.109100    306.407074
p

info retrieval:  42%|████▏     | 188/443 [1:45:47<2:43:33, 38.48s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss       10.555143     11.436027
postjump_num_loss     388.497620    422.772736
prejump_diag_loss       0.001051      0.000767
postjump_diag_loss      0.001053      0.000789
num_loss               48.349391     52.569697
diag_loss               0.001051      0.000769
ode_loss                0.049399      0.053338
l1_loss             13942.534180  13942.534180
l1_loss_per_point      23.997477     42.768510
l2_loss              1751.088257   1751.088257
l2_loss_per_point       3.013921      5.371436
dyn_loss                 18238.0       10582.0
dyn_loss_per_week      24.698393       9.38834
loss                 0.049752098    0.05373198
INFO:ode:
                          Training   Valdation
accuracy                  0.788028    0.783231
recall                    0.796429    0.876923
npv                       0.986775    0.995377
s